[View in Colaboratory](https://colab.research.google.com/github/assaflehr/language-style-transfer/blob/master/notebooks/ST_EXP1_sentence_autoendcoder.ipynb)

Exp1 report:

First check it's working at all (use 160 examples) with short-sentences(<12 words).  Took few hours to find initial working model. encoder-decoder based on simple (one-directional) CudaGRU. a lot of code taken from tf eager [tutorial](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/contrib/eager/python/examples/nmt_with_attention/nmt_with_attention.ipynb#scrollTo=cnxC7q-j3jFD)
num_examples = 160 , crop_sentence_max_words=12 , BATCH_SIZE = 64 , embedding_dim = 256 , units = 1024 


Then increase to 16000 examples (all file) and let it run


In [1]:
# Try experimenting with the size of that dataset
num_examples = 16000 #16000  #max = 14716+3679 in current file
crop_sentence_max_words=12
BATCH_SIZE = 64
embedding_dim = 256
units = 1024  #hidden size in encoder and decoder (as one pass to the other)


### Exp 1 



In [3]:
from __future__ import absolute_import, division, print_function

# Import TensorFlow >= 1.9 and enable eager execution
import tensorflow as tf
print(tf.__version__)
tf.enable_eager_execution()



1.8.0


ValueError: tf.enable_eager_execution must be called at program startup.

In [5]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import time


#if  tf.test.gpu_device_name() != '/device:GPU:0':
#  raise SystemError('GPU device not found')



# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w,crop_sentence_max_words):
    w = unicode_to_ascii(w.lower().strip())
    
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ." 
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    
    w = w.rstrip().strip()
    
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w= ' '.join(w.split(' ')[:crop_sentence_max_words])
    w = '<start> ' + w + ' <end>'
    
    return w
 
# 1. Remove the accents
# 2. Clean the sentences
# 3. Return sentence list
def create_dataset(path, num_examples,crop_sentence_max_words,verbose=True):
    lines = open(path, encoding='UTF-8').read().strip().split('\n')
    sentences = [preprocess_sentence(s,crop_sentence_max_words) for s  in lines[:num_examples]]
    return sentences
  
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
  def __init__(self, lang):
    self.lang = lang
    self.word2idx = {}
    self.idx2word = {}
    self.vocab = set()
    
    self.create_index()
    
  def create_index(self):
    for phrase in self.lang:
      self.vocab.update(phrase.split(' '))
    
    self.vocab = sorted(self.vocab)
    
    self.word2idx['<pad>'] = 0
    for index, word in enumerate(self.vocab):
      self.word2idx[word] = index + 1
    
    for word, index in self.word2idx.items():
      self.idx2word[index] = word
      
      
def max_length(tensor):
    return max(len(t) for t in tensor)


def load_dataset(path, num_examples,crop_sentence_max_words=9999,verbose=True):
    # creating cleaned input, output pairs
    sentences = create_dataset(path, num_examples,crop_sentence_max_words)
    if verbose:
      print (sentences[:2])

    # index language using the class defined above    
    targ_lang = LanguageIndex(sentences)
    
    # Vectorize the input and target languages
    
    target_tensor = [[targ_lang.word2idx[s] for s in en.split(' ')] for en in sentences]
    
    # Calculate max_length of input and output tensor
    # Here, we'll set those to the longest sentence in the dataset
    max_length_tar = max_length(target_tensor)
      
    # Padding the input and output tensor to the maximum length
    target_tensor = tf.keras.preprocessing.sequence.pad_sequences(target_tensor, 
                                                                  maxlen=max_length_tar, 
                                                                  padding='post')
    
    return  target_tensor,  targ_lang,  max_length_tar
  
  # Download the file
path_to_file = tf.keras.utils.get_file(
    'train.modern.nltktok', origin='https://raw.githubusercontent.com/harsh19/Shakespearizing-Modern-English/master/data/train.modern.nltktok',
    extract=False)
print (path_to_file)


target_tensor,  targ_lang,  max_length_targ = load_dataset(path_to_file, num_examples,crop_sentence_max_words=crop_sentence_max_words)
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(target_tensor, target_tensor, test_size=0.2)

  
# Show length
len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val),'max_length_targ',max_length_targ
  



925696/918924 [==============================] - 1s 1us/step
C:\Users\Assaf\.keras\datasets\train.modern.nltktok
['<start> i have half a mind to hit you before you speak again <end>', '<start> but if antony is alive , healthy , friendly with caesar , <end>']


(12800, 12800, 3200, 3200, 'max_length_targ', 14)

### model

In [26]:
def gru(units):
  # If you have a GPU, we recommend using CuDNNGRU(provides a 3x speedup than GRU)
  # the code automatically does that.
  if tf.test.is_gpu_available():
    print ('using CuDNNGRU')
    return tf.keras.layers.CuDNNGRU(units, 
                                    return_sequences=True, 
                                    return_state=True, 
                                    recurrent_initializer='glorot_uniform')
  else:
    print ('using CPU')
    return tf.keras.layers.GRU(units, 
                               return_sequences=True, 
                               return_state=True, 
                               recurrent_activation='sigmoid', 
                               recurrent_initializer='glorot_uniform')

  
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.enc_units)
        
    def call(self, x, hidden):
        """
        """
        x = self.embedding(x)
        outputs, states = self.gru(x, initial_state = hidden)        
        return outputs, states
    
    def get_initial_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))  

      
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = gru(self.dec_units)
        self.fc = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x, hidden):
        # enc_output shape == (batch_size, max_length, hidden_size)
        
        # hidden shape == (batch_size, hidden size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden size)
        # we are doing this to perform addition to calculate the score
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        
        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)
        
        # passing the concatenated vector to the GRU
        output, new_hidden = self.gru(x, initial_state = hidden)
        
        # output shape == (batch_size * max_length, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))
        
        # output shape == (batch_size * max_length, vocab)
        x = self.fc(output)
        
        return x,new_hidden
        
    #def get_initial_hidden_state(self):
    #    return tf.zeros((self.batch_sz, self.dec_units)) 


BUFFER_SIZE = len(input_tensor_train)

vocab_tar_size = len(targ_lang.word2idx)
vocab_inp_size= vocab_tar_size 

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

      
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)      

AttributeError: 'SymbolDatabase' object has no attribute 'RegisterServiceDescriptor'

## EVAL SINGLE

In [23]:
verbose=False

def vis_tensor(name,t):
  if verbose:
    cnt=5
    content = (t[0][:cnt].numpy() if len(t.shape.dims)<=2 else t[0][0][:cnt].numpy())
    print (name, t.shape, content),#tf.reduce_max(t).numpy(),tf.reduce_mean(t).numpy(),tf.reduce_min(t).numpy())
    print 
    #print ('\n')

def evaluate_single(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    #attention_plot = np.zeros((max_length_targ, max_length_inp))
    
    sentence = preprocess_sentence(sentence,crop_sentence_max_words)

    inputs = [inp_lang.word2idx[i] for i in sentence.split(' ')]
    inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen=max_length_inp, padding='post')
    inputs = tf.convert_to_tensor(inputs)
    vis_tensor ('inputs',inputs)
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)
    vis_tensor ('enc_hidden',enc_hidden)
    vis_tensor ('enc_out',enc_out)
    
    
    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word2idx['<start>']], 0)

    for t in range(max_length_targ):
        predictions, dec_hidden = decoder(dec_input, dec_hidden)
        
        predicted_id = tf.multinomial(tf.exp(predictions), num_samples=1)[0][0].numpy()

        result += targ_lang.idx2word[predicted_id] + ' '

        if targ_lang.idx2word[predicted_id] == '<end>':
            return result, sentence
        
        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 0)

    return result, sentence
  
def translate(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
    result, sentence = evaluate_single(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ)
        
    print('Input:      {}'.format(sentence))
    print('Predicted : {}'.format(result))
print (vocab_inp_size)   


translate("I haven't done anything to you .", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)
print ('\n\n\n')
#translate("Good madam , restrain yourself .", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)
#translate("life is good .", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)
translate("Good madam , restrain yourself", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)

translate("Good madam , I haven't done anything to you", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)

translate("Ho my god, what happened.", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)

6928
Input:      <start> i haven t done anything to you . <end>
Predicted : edges younger goat proven eros occupied tries crests disorderly shocking address utterly hag surprised 




Input:      <start> good madam , restrain yourself <end>
Predicted : freckled wine mustard branches where lead forty playwrights waste syracusian hamlet york ransom speechless 
Input:      <start> good madam , i haven t done anything to you <end>
Predicted : afterward spent partying wicked humor yield exposed chewing unbelievable heat want foulest final warms 
Input:      <start> ho my god , what happened . <end>
Predicted : able rule eats bottom companions unprofitable sweetly resisted herself intervenes inflict lecherous instantly pulpit 


## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [25]:

verbose=True
def vis_tensor(name,t,verbose):
  if verbose:
    cnt=5
    content = (t[0][:cnt].numpy() if len(t.shape.dims)<=2 else t[0][0][:cnt].numpy())
    print (name, t.shape, content),#tf.reduce_max(t).numpy(),tf.reduce_mean(t).numpy(),tf.reduce_min(t).numpy())
    print 
    #print ('\n')
    
#print('Found GPU at: {}'.format(device_name))

optimizer = tf.train.AdamOptimizer()

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=real, logits=pred) * mask
  return tf.reduce_mean(loss_)


EPOCHS = 200

for epoch in range(EPOCHS):
    start = time.time()
    
    hidden = encoder.get_initial_hidden_state()
    total_loss = 0
    
    for (batch, (inp, targ)) in enumerate(dataset):
        print (batch,inp.shape,targ.shape)
        loss = 0
        verbose = True if batch<1 else False

        with tf.GradientTape() as tape:
            vis_tensor('inp',inp,verbose)
            vis_tensor('hidden',hidden,verbose)
            enc_output, enc_hidden = encoder(inp, hidden)
            vis_tensor('enc_output',enc_output)
            vis_tensor('enc_hidden',enc_hidden)
            
            dec_hidden = enc_hidden
            
            
            dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       
            
            # Teacher forcing - feeding the target as the next input
            for t in range(1, targ.shape[1]):
                # passing enc_output to the decoder
                vis_tensor('dec_input',dec_input)
                vis_tensor('dec_hidden',dec_hidden)
                predictions, dec_hidden = decoder(dec_input, dec_hidden)
                vis_tensor('predictions',predictions)
                vis_tensor('dec_hidden',dec_hidden)
                
                loss += loss_function(targ[:, t], predictions)
                
                # using teacher forcing, TODO: change it to only sometimes run
                dec_input = tf.expand_dims(targ[:, t], 1)
                
           
        total_loss += (loss / int(targ.shape[1]))
        
        variables = encoder.variables + decoder.variables
        
        gradients = tape.gradient(loss, variables)
      
        optimizer.apply_gradients(zip(gradients, variables), tf.train.get_or_create_global_step())

        if batch % 10 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         loss.numpy() / int(targ.shape[1])))
    
    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                        total_loss/len(target_tensor)))
    translate("I haven't done anything to you .", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)
    translate("We try to see how you are doing", encoder, decoder, targ_lang, targ_lang, max_length_targ, max_length_targ)

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

0 (14,) (14,)


InvalidArgumentError: Index out of range using input dim 0; input has only 0 dims [Op:StridedSlice] name: strided_slice/

## TODO: Eval


In [0]:
## TODO: Eval

In [10]:
def eval(encoder, decoder, lang, max_length,val_dataset):
  """
    evaluate total-loss on one EPOC of the val_dataset
  """

  
  hidden = encoder.get_initial_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset): #dataset return (#batch,(x,y)
      loss = 0

      _ , enc_hidden = encoder(inp, hidden)

      dec_hidden = enc_hidden
      
      #
      dec_input = tf.expand_dims([targ_lang.word2idx['<start>']] * BATCH_SIZE, 1)       

      # Teacher forcing - feeding the target as the next input
      for t in range(1, targ.shape[1]):
          # passing enc_output to the decoder
          predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)

          loss += loss_function(targ[:, t], predictions)

          # using teacher forcing, TODO: change it to only sometimes run
          dec_input = tf.expand_dims(targ[:, t], 1)

      total_loss += (loss / int(targ.shape[1]))


val_dataset = tf.data.Dataset.from_tensor_slices((target_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
val_dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))
eval(encoder, decoder, lang, max_length,val_dataset)

NameError: ignored

In [21]:
val_dataset = tf.data.Dataset.from_tensor_slices((target_tensor_val, target_tensor_val)).shuffle(BUFFER_SIZE)
#val_dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(BATCH_SIZE))

for (batch, (inp, targ)) in enumerate(val_dataset):
  print ('batch',batch,'inp',inp.shape,'targ',targ.shape)
dataset= val_dataset

batch 0 inp (14,) targ (14,)
batch 1 inp (14,) targ (14,)
batch 2 inp (14,) targ (14,)
batch 3 inp (14,) targ (14,)
batch 4 inp (14,) targ (14,)
batch 5 inp (14,) targ (14,)
batch 6 inp (14,) targ (14,)
batch 7 inp (14,) targ (14,)
batch 8 inp (14,) targ (14,)
batch 9 inp (14,) targ (14,)
batch 10 inp (14,) targ (14,)
batch 11 inp (14,) targ (14,)
batch 12 inp (14,) targ (14,)
batch 13 inp (14,) targ (14,)
batch 14 inp (14,) targ (14,)
batch 15 inp (14,) targ (14,)
batch 16 inp (14,) targ (14,)
batch 17 inp (14,) targ (14,)
batch 18 inp (14,) targ (14,)
batch 19 inp (14,) targ (14,)
batch 20 inp (14,) targ (14,)
batch 21 inp (14,) targ (14,)
batch 22 inp (14,) targ (14,)
batch 23 inp (14,) targ (14,)
batch 24 inp (14,) targ (14,)
batch 25 inp (14,) targ (14,)
batch 26 inp (14,) targ (14,)
batch 27 inp (14,) targ (14,)
batch 28 inp (14,) targ (14,)
batch 29 inp (14,) targ (14,)
batch 30 inp (14,) targ (14,)
batch 31 inp (14,) targ (14,)
batch 32 inp (14,) targ (14,)
batch 33 inp (14,) t

batch 410 inp (14,) targ (14,)
batch 411 inp (14,) targ (14,)
batch 412 inp (14,) targ (14,)
batch 413 inp (14,) targ (14,)
batch 414 inp (14,) targ (14,)
batch 415 inp (14,) targ (14,)
batch 416 inp (14,) targ (14,)
batch 417 inp (14,) targ (14,)
batch 418 inp (14,) targ (14,)
batch 419 inp (14,) targ (14,)
batch 420 inp (14,) targ (14,)
batch 421 inp (14,) targ (14,)
batch 422 inp (14,) targ (14,)
batch 423 inp (14,) targ (14,)
batch 424 inp (14,) targ (14,)
batch 425 inp (14,) targ (14,)
batch 426 inp (14,) targ (14,)
batch 427 inp (14,) targ (14,)
batch 428 inp (14,) targ (14,)
batch 429 inp (14,) targ (14,)
batch 430 inp (14,) targ (14,)
batch 431 inp (14,) targ (14,)
batch 432 inp (14,) targ (14,)
batch 433 inp (14,) targ (14,)
batch 434 inp (14,) targ (14,)
batch 435 inp (14,) targ (14,)
batch 436 inp (14,) targ (14,)
batch 437 inp (14,) targ (14,)
batch 438 inp (14,) targ (14,)
batch 439 inp (14,) targ (14,)
batch 440 inp (14,) targ (14,)
batch 441 inp (14,) targ (14,)
batch 44

batch 784 inp (14,) targ (14,)
batch 785 inp (14,) targ (14,)
batch 786 inp (14,) targ (14,)
batch 787 inp (14,) targ (14,)
batch 788 inp (14,) targ (14,)
batch 789 inp (14,) targ (14,)
batch 790 inp (14,) targ (14,)
batch 791 inp (14,) targ (14,)
batch 792 inp (14,) targ (14,)
batch 793 inp (14,) targ (14,)
batch 794 inp (14,) targ (14,)
batch 795 inp (14,) targ (14,)
batch 796 inp (14,) targ (14,)
batch 797 inp (14,) targ (14,)
batch 798 inp (14,) targ (14,)
batch 799 inp (14,) targ (14,)
batch 800 inp (14,) targ (14,)
batch 801 inp (14,) targ (14,)
batch 802 inp (14,) targ (14,)
batch 803 inp (14,) targ (14,)
batch 804 inp (14,) targ (14,)
batch 805 inp (14,) targ (14,)
batch 806 inp (14,) targ (14,)
batch 807 inp (14,) targ (14,)
batch 808 inp (14,) targ (14,)
batch 809 inp (14,) targ (14,)
batch 810 inp (14,) targ (14,)
batch 811 inp (14,) targ (14,)
batch 812 inp (14,) targ (14,)
batch 813 inp (14,) targ (14,)
batch 814 inp (14,) targ (14,)
batch 815 inp (14,) targ (14,)
batch 81

batch 1174 inp (14,) targ (14,)
batch 1175 inp (14,) targ (14,)
batch 1176 inp (14,) targ (14,)
batch 1177 inp (14,) targ (14,)
batch 1178 inp (14,) targ (14,)
batch 1179 inp (14,) targ (14,)
batch 1180 inp (14,) targ (14,)
batch 1181 inp (14,) targ (14,)
batch 1182 inp (14,) targ (14,)
batch 1183 inp (14,) targ (14,)
batch 1184 inp (14,) targ (14,)
batch 1185 inp (14,) targ (14,)
batch 1186 inp (14,) targ (14,)
batch 1187 inp (14,) targ (14,)
batch 1188 inp (14,) targ (14,)
batch 1189 inp (14,) targ (14,)
batch 1190 inp (14,) targ (14,)
batch 1191 inp (14,) targ (14,)
batch 1192 inp (14,) targ (14,)
batch 1193 inp (14,) targ (14,)
batch 1194 inp (14,) targ (14,)
batch 1195 inp (14,) targ (14,)
batch 1196 inp (14,) targ (14,)
batch 1197 inp (14,) targ (14,)
batch 1198 inp (14,) targ (14,)
batch 1199 inp (14,) targ (14,)
batch 1200 inp (14,) targ (14,)
batch 1201 inp (14,) targ (14,)
batch 1202 inp (14,) targ (14,)
batch 1203 inp (14,) targ (14,)
batch 1204 inp (14,) targ (14,)
batch 12

batch 1549 inp (14,) targ (14,)
batch 1550 inp (14,) targ (14,)
batch 1551 inp (14,) targ (14,)
batch 1552 inp (14,) targ (14,)
batch 1553 inp (14,) targ (14,)
batch 1554 inp (14,) targ (14,)
batch 1555 inp (14,) targ (14,)
batch 1556 inp (14,) targ (14,)
batch 1557 inp (14,) targ (14,)
batch 1558 inp (14,) targ (14,)
batch 1559 inp (14,) targ (14,)
batch 1560 inp (14,) targ (14,)
batch 1561 inp (14,) targ (14,)
batch 1562 inp (14,) targ (14,)
batch 1563 inp (14,) targ (14,)
batch 1564 inp (14,) targ (14,)
batch 1565 inp (14,) targ (14,)
batch 1566 inp (14,) targ (14,)
batch 1567 inp (14,) targ (14,)
batch 1568 inp (14,) targ (14,)
batch 1569 inp (14,) targ (14,)
batch 1570 inp (14,) targ (14,)
batch 1571 inp (14,) targ (14,)
batch 1572 inp (14,) targ (14,)
batch 1573 inp (14,) targ (14,)
batch 1574 inp (14,) targ (14,)
batch 1575 inp (14,) targ (14,)
batch 1576 inp (14,) targ (14,)
batch 1577 inp (14,) targ (14,)
batch 1578 inp (14,) targ (14,)
batch 1579 inp (14,) targ (14,)
batch 15

batch 1906 inp (14,) targ (14,)
batch 1907 inp (14,) targ (14,)
batch 1908 inp (14,) targ (14,)
batch 1909 inp (14,) targ (14,)
batch 1910 inp (14,) targ (14,)
batch 1911 inp (14,) targ (14,)
batch 1912 inp (14,) targ (14,)
batch 1913 inp (14,) targ (14,)
batch 1914 inp (14,) targ (14,)
batch 1915 inp (14,) targ (14,)
batch 1916 inp (14,) targ (14,)
batch 1917 inp (14,) targ (14,)
batch 1918 inp (14,) targ (14,)
batch 1919 inp (14,) targ (14,)
batch 1920 inp (14,) targ (14,)
batch 1921 inp (14,) targ (14,)
batch 1922 inp (14,) targ (14,)
batch 1923 inp (14,) targ (14,)
batch 1924 inp (14,) targ (14,)
batch 1925 inp (14,) targ (14,)
batch 1926 inp (14,) targ (14,)
batch 1927 inp (14,) targ (14,)
batch 1928 inp (14,) targ (14,)
batch 1929 inp (14,) targ (14,)
batch 1930 inp (14,) targ (14,)
batch 1931 inp (14,) targ (14,)
batch 1932 inp (14,) targ (14,)
batch 1933 inp (14,) targ (14,)
batch 1934 inp (14,) targ (14,)
batch 1935 inp (14,) targ (14,)
batch 1936 inp (14,) targ (14,)
batch 19

batch 2250 inp (14,) targ (14,)
batch 2251 inp (14,) targ (14,)
batch 2252 inp (14,) targ (14,)
batch 2253 inp (14,) targ (14,)
batch 2254 inp (14,) targ (14,)
batch 2255 inp (14,) targ (14,)
batch 2256 inp (14,) targ (14,)
batch 2257 inp (14,) targ (14,)
batch 2258 inp (14,) targ (14,)
batch 2259 inp (14,) targ (14,)
batch 2260 inp (14,) targ (14,)
batch 2261 inp (14,) targ (14,)
batch 2262 inp (14,) targ (14,)
batch 2263 inp (14,) targ (14,)
batch 2264 inp (14,) targ (14,)
batch 2265 inp (14,) targ (14,)
batch 2266 inp (14,) targ (14,)
batch 2267 inp (14,) targ (14,)
batch 2268 inp (14,) targ (14,)
batch 2269 inp (14,) targ (14,)
batch 2270 inp (14,) targ (14,)
batch 2271 inp (14,) targ (14,)
batch 2272 inp (14,) targ (14,)
batch 2273 inp (14,) targ (14,)
batch 2274 inp (14,) targ (14,)
batch 2275 inp (14,) targ (14,)
batch 2276 inp (14,) targ (14,)
batch 2277 inp (14,) targ (14,)
batch 2278 inp (14,) targ (14,)
batch 2279 inp (14,) targ (14,)
batch 2280 inp (14,) targ (14,)
batch 22

batch 2553 inp (14,) targ (14,)
batch 2554 inp (14,) targ (14,)
batch 2555 inp (14,) targ (14,)
batch 2556 inp (14,) targ (14,)
batch 2557 inp (14,) targ (14,)
batch 2558 inp (14,) targ (14,)
batch 2559 inp (14,) targ (14,)
batch 2560 inp (14,) targ (14,)
batch 2561 inp (14,) targ (14,)
batch 2562 inp (14,) targ (14,)
batch 2563 inp (14,) targ (14,)
batch 2564 inp (14,) targ (14,)
batch 2565 inp (14,) targ (14,)
batch 2566 inp (14,) targ (14,)
batch 2567 inp (14,) targ (14,)
batch 2568 inp (14,) targ (14,)
batch 2569 inp (14,) targ (14,)
batch 2570 inp (14,) targ (14,)
batch 2571 inp (14,) targ (14,)
batch 2572 inp (14,) targ (14,)
batch 2573 inp (14,) targ (14,)
batch 2574 inp (14,) targ (14,)
batch 2575 inp (14,) targ (14,)
batch 2576 inp (14,) targ (14,)
batch 2577 inp (14,) targ (14,)
batch 2578 inp (14,) targ (14,)
batch 2579 inp (14,) targ (14,)
batch 2580 inp (14,) targ (14,)
batch 2581 inp (14,) targ (14,)
batch 2582 inp (14,) targ (14,)
batch 2583 inp (14,) targ (14,)
batch 25

batch 2935 inp (14,) targ (14,)
batch 2936 inp (14,) targ (14,)
batch 2937 inp (14,) targ (14,)
batch 2938 inp (14,) targ (14,)
batch 2939 inp (14,) targ (14,)
batch 2940 inp (14,) targ (14,)
batch 2941 inp (14,) targ (14,)
batch 2942 inp (14,) targ (14,)
batch 2943 inp (14,) targ (14,)
batch 2944 inp (14,) targ (14,)
batch 2945 inp (14,) targ (14,)
batch 2946 inp (14,) targ (14,)
batch 2947 inp (14,) targ (14,)
batch 2948 inp (14,) targ (14,)
batch 2949 inp (14,) targ (14,)
batch 2950 inp (14,) targ (14,)
batch 2951 inp (14,) targ (14,)
batch 2952 inp (14,) targ (14,)
batch 2953 inp (14,) targ (14,)
batch 2954 inp (14,) targ (14,)
batch 2955 inp (14,) targ (14,)
batch 2956 inp (14,) targ (14,)
batch 2957 inp (14,) targ (14,)
batch 2958 inp (14,) targ (14,)
batch 2959 inp (14,) targ (14,)
batch 2960 inp (14,) targ (14,)
batch 2961 inp (14,) targ (14,)
batch 2962 inp (14,) targ (14,)
batch 2963 inp (14,) targ (14,)
batch 2964 inp (14,) targ (14,)
batch 2965 inp (14,) targ (14,)
batch 29